In [2]:
import numpy as np
import pandas as pd
import pandas as pd
from sklearn.preprocessing import StandardScaler
from sklearn.metrics.pairwise import cosine_similarity

# === 1. Load Raw Excel ===
df = pd.read_excel("../data/2024_Finaccess_Publicdata.xlsx")

In [1]:


# === 2. Select Relevant Columns ===
relevant_cols = [
    "A08",              # area_type (Urban/Rural)
    "A13",              # gender
    "B3Ii",             # monthly_income
    "U23",              # monthly_expenditure
    "C1_1a",            # save_bank
    "C1_2",             # save_mobile_money
    "C1_4",             # save_sacco
    "C1_6",             # save_friends
    "C1_9",             # save_digital
    "C1_15",            # loan_mobile
    "C1_17",            # loan_sacco
    "C1_19",            # loan_digital
    "C1_25",            # loan_family
    "C1_35"             # invest_forex
]
df_subset = df[relevant_cols].copy()

# === 3. Rename Columns ===
df_subset.columns = [
    "area_type", "gender", "monthly_income", "monthly_expenditure",
    "save_bank", "save_mobile_money", "save_sacco", "save_friends", "save_digital",
    "loan_mobile", "loan_sacco", "loan_digital", "loan_family", "invest_forex"
]


NameError: name 'df' is not defined

In [4]:

# === 4. Map Usage Levels to Ordinal ===
usage_map = {
    "Never used": 0,
    "Used to use": 1,
    "Currently use": 2,
    pd.NA: 0,
    None: 0
}
behavior_cols = [
    'save_bank', 'save_mobile_money', 'save_sacco', 'save_friends', 'save_digital',
    'loan_mobile', 'loan_sacco', 'loan_digital', 'loan_family', 'invest_forex'
]
df_subset[behavior_cols] = df_subset[behavior_cols].applymap(lambda x: usage_map.get(x, 0))


C:\Users\ronny somi\AppData\Local\Temp\ipykernel_8400\1233527928.py:13: FutureWarning: DataFrame.applymap has been deprecated. Use DataFrame.map instead.
  df_subset[behavior_cols] = df_subset[behavior_cols].applymap(lambda x: usage_map.get(x, 0))


In [5]:

# === 5. Encode Demographics ===
df_subset['gender'] = df_subset['gender'].map({'Male': 0, 'Female': 1})
df_subset['area_type'] = df_subset['area_type'].map({'Rural': 0, 'Urban': 1})

In [6]:


# === 6. Fill Missing Numeric Data ===
df_subset['monthly_income'] = df_subset['monthly_income'].fillna(df_subset['monthly_income'].median())
df_subset['monthly_expenditure'] = df_subset['monthly_expenditure'].fillna(df_subset['monthly_expenditure'].median())

# === 7. Scale Features ===
scaler = StandardScaler()
household_scaled = scaler.fit_transform(df_subset)


In [7]:
# === 8. Create 3 Dummy Investment Profiles ===
investment_profiles = pd.DataFrame([
    [1, 1, 0.3, 0.3, 2, 1, 2, 1, 0, 0, 0, 0, 0, 0],  # Conservative
    [1, 1, 0.5, 0.5, 1, 1, 1, 1, 1, 1, 1, 1, 1, 0],  # Balanced
    [1, 1, 0.6, 0.6, 0, 2, 0, 0, 2, 2, 2, 2, 2, 2]   # Aggressive
], columns=df_subset.columns, index=['conservative', 'balanced', 'aggressive'])

investment_scaled = scaler.transform(investment_profiles)

In [8]:


# === 9. Compute Cosine Similarity ===
similarity_matrix = cosine_similarity(household_scaled, investment_scaled)
recommendations = pd.DataFrame(similarity_matrix, columns=investment_profiles.index)

# === 10. Label Top-1 Strategy ===
df_subset["investment_label"] = recommendations.apply(lambda row: row.nlargest(1).index[0], axis=1)

In [9]:


# === 11. Save to CSV ===
df_subset.to_csv("labeled_household_investments.csv", index=False)
print("✅ Saved to labeled_household_investments.csv")


✅ Saved to labeled_household_investments.csv
